In [ ]:
import tables
import pandas as pd
import numpy as np
def read_data_tables(data_path, cycle):
        try:
            with tables.open_file(data_path, mode='r') as file:
                data_node = file.get_node(f"/Cycle{cycle:04d}/block0_values")
                data_array = data_node[:]  # Exclude the first column
                columns_node = file.get_node(f"/Cycle{cycle:04d}/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                return pd.DataFrame(data_array, columns=columns)
        except Exception as e:
            print(e)
            return None
def read_capacity_tables(data_path, cycle):
        try:
            with tables.open_file(data_path, mode='r') as file:
                data_node = file.get_node(f"/capacity{cycle:04d}/block0_values")
                data_array = data_node[:]  # Exclude the first column
                columns_node = file.get_node(f"/capacity{cycle:04d}/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                return pd.DataFrame(data_array, columns=columns)
        except Exception as e:
            print(e)
            return None
    # Function to read SoH using PyTables
def read_soh_tables(data_path):
        try:
            with tables.open_file(data_path, mode='r') as file:
                soh_node = file.get_node("/Info/block0_values")
                soh_array = soh_node[:]  # Exclude the first column
                columns_node = file.get_node("/Info/block0_items")
                columns = columns_node[:].astype(str) # Exclude the first column name
                soh_data = soh_array[:, columns == 'real_capacity'].flatten()
                soh_data = soh_data / soh_data[0]
                return pd.Series(soh_data)
        except Exception as e:
            return None

cycle = 0
file_path = "Synthetic_data/episode_963738d2-574a-4461-9c00-598a47b8557a.result"
data = read_data_tables(file_path, cycle)
soh_data = read_soh_tables(file_path)

if data is None or soh_data is None or len(data) == 0:
    raise IndexError(
        f"No valid data found for file {file_path}, cycle {cycle}"
    )

cycle_data = data[data["Step"] == data["Step"].max()][
    ["V", "C", "SoC", "VoC", "relative_time"]
]
if cycle_data.empty:
    raise IndexError(
        f"Cycle data is empty for file {file_path}, cycle {cycle}"
    )
time_vector = cycle_data["relative_time"].values


# Measure the end time for resampling

# Extract resampled values
V = np.array(cycle_data["V"], dtype=np.float32)
C = np.array(cycle_data["C"], dtype=np.float32)
SoC = np.array(cycle_data["SoC"], dtype=np.float32)
VoC = np.array(cycle_data["VoC"], dtype=np.float32)
# Compute time difference (dt) efficiently
dt = np.diff(
    cycle_data["relative_time"],
    prepend=cycle_data["relative_time"].iloc[0],
)


In [23]:
soh_data

0    1.0
dtype: float64